In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf
%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

2023-10-27 10:58:00.580573: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def add_location(frame, location):
    frame["location"] = location
    return frame

In [3]:
def removeNaN(frame):
    frame = frame.fillna(method="ffill")
    frame = frame.fillna(method="bfill")
    frame = frame.fillna(value=0)
    return frame

In [4]:
def averageFeatures(frame):
    frame["date_forecast"] = frame["date_forecast"].apply(lambda a : a.replace(minute=0))
    return frame.groupby(["date_forecast"])[frame.iloc[:,1:].columns].mean().reset_index()

Load and clean data

In [73]:
train_a = pd.read_parquet('A/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')

train_b = pd.read_parquet('B/train_targets.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')

train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

train_a = add_location(train_a, "A")
train_b = add_location(train_b, "B")
train_c = add_location(train_c, "C")
X_train_estimated_a = X_train_estimated_a.iloc[:,1:]
X_train_estimated_b = X_train_estimated_b.iloc[:,1:]
X_train_estimated_c = X_train_estimated_c.iloc[:,1:]
X_train_estimated_a = add_location(averageFeatures(removeNaN(X_train_estimated_a)), "A")
X_train_estimated_b = add_location(averageFeatures(removeNaN(X_train_estimated_b)), "B")
X_train_estimated_c = add_location(averageFeatures(removeNaN(X_train_estimated_c)), "C")
X_train_observed_a = add_location(averageFeatures(removeNaN(X_train_observed_a)), "A")
X_train_observed_b = add_location(averageFeatures(removeNaN(X_train_observed_b)), "B")
X_train_observed_c = add_location(averageFeatures(removeNaN(X_train_observed_c)), "C")

/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="bfill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = f

In [509]:
#Beklager ekstremt nasty kode her


targets = pd.concat([train_a, train_b, train_c]).sample(frac=1)
features = pd.concat([
                     X_train_observed_a, 
                     X_train_observed_b, 
                     X_train_observed_c,
                     X_train_estimated_a,
                    X_train_estimated_b,
                    X_train_estimated_c]).sample(frac=1)

train_features = features.iloc[:90000,:]
train_features = train_features.rename(columns={"date_forecast" : "time"})
trainset = pd.merge(train_features, targets, how="right", on=["time","location"]).dropna()

eval_features = features.iloc[90000:,:]
eval_features = eval_features.rename(columns={"date_forecast" : "time"})

evalset = pd.merge(eval_features, targets, how="right", on=["time","location"]).dropna()

trainsetX = trainset.iloc[:, :-1].reset_index().iloc[:,1:]
trainsetX["day"] = trainsetX["time"].dt.day_of_year
trainsetX["hour"] = trainsetX["time"].dt.hour


sample567 = trainsetX["time"].apply(lambda time : time.month in[5, 6, 7])
#display(sample_weights)



trainsetX = trainsetX.iloc[:,1:]
trainsetX["location"] = trainsetX["location"].map({"A": 0, "B": 1, "C": 2})


dataMean = trainsetX.mean()
dataStd = trainsetX.std()
trainsetX = ((trainsetX-dataMean)/dataStd).fillna(value=0)
trainsetY = trainset.iloc[:, -1:].reset_index().iloc[:,1:]


evalsetX = evalset.iloc[:, :-1].reset_index().iloc[:, 1:]

evalsetX["day"] = evalsetX["time"].dt.day_of_year
evalsetX["hour"] = evalsetX["time"].dt.hour


eval567 = evalsetX["time"].apply(lambda time : time.month in[5, 6, 7])

evalsetX = evalsetX.iloc[:,1:]
evalsetX["location"] = evalsetX["location"].map({"A": 0, "B": 1, "C": 2})
evalsetX = ((evalsetX-dataMean)/dataStd).fillna(value=0)
evalsetY = evalset.iloc[:, -1:].reset_index().iloc[:, 1:]





In [518]:
model = tf.keras.models.Sequential([
  tf.keras.layers.GaussianNoise(stddev=0.03, seed=42),
  tf.keras.layers.Dense(40, activation="tanh"),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(15, activation="relu"),
  tf.keras.layers.Dense(5, activation="relu")
])


In [520]:
#weighting for samples from months matching the testset
sample_weights = sample567.map({True : 1.35, False : 1})

model.compile(
  optimizer=tf.keras.optimizers.experimental.Adadelta(learning_rate=0.5),
  loss='mean_absolute_error'
  )

history = model.fit(trainsetX, 
                    trainsetY, 
                    batch_size=500, 
                    epochs=300,
                    validation_data=(evalsetX, evalsetY),
                    sample_weight=sample_weights
                    )

Epoch 1/300
166/166 [==============================] - 3s 6ms/step - loss: 88.4878 - val_loss: 85.0789
Epoch 2/300
166/166 [==============================] - 1s 4ms/step - loss: 88.5514 - val_loss: 84.9581
Epoch 3/300
166/166 [==============================] - 1s 4ms/step - loss: 88.4875 - val_loss: 85.0324
Epoch 4/300
166/166 [==============================] - 1s 4ms/step - loss: 88.3614 - val_loss: 84.8789
Epoch 5/300
166/166 [==============================] - 1s 4ms/step - loss: 88.4626 - val_loss: 85.3372
Epoch 6/300
166/166 [==============================] - 1s 4ms/step - loss: 88.3065 - val_loss: 84.9765
Epoch 7/300
166/166 [==============================] - 1s 4ms/step - loss: 88.3685 - val_loss: 85.1215
Epoch 8/300
166/166 [==============================] - 1s 4ms/step - loss: 88.3673 - val_loss: 85.1421
Epoch 9/300
166/166 [==============================] - 1s 4ms/step - loss: 88.3223 - val_loss: 85.0977
Epoch 10/300
166/166 [==============================] - 1s 4ms/step - los

In [515]:

evalset567 = evalsetX.join(evalsetY)
evalset567 = evalset567[eval567]
evalset567X = evalset567.iloc[:,:-1]
evalset567Y = evalset567.iloc[:,-1:]

#results only in may, june, july(since the test set only contains data from those months)
results1 = model.evaluate(evalset567X, evalset567Y)

#results for entire eval set
results2 = model.evaluate(evalsetX, evalsetY)

326/326 [==============================] - 1s 2ms/step - loss: 84.6936


In [13]:
#lagre 

preds = model.predict(evalsetX)
compare = pd.concat([evalsetY, pd.DataFrame(preds)], axis=1)
compare.to_csv("test1.csv")

118/118 [==============================] - 1s 5ms/step


In [173]:
#Igjen beklager for nasty kode

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('A/X_test_estimated.parquet')

X_test_estimated_a = X_test_estimated_a.iloc[:,1:]
X_test_estimated_b = X_test_estimated_b.iloc[:,1:]
X_test_estimated_c = X_test_estimated_c.iloc[:,1:]
X_test_estimated_a = add_location(averageFeatures(removeNaN(X_test_estimated_a)), "A")
X_test_estimated_b = add_location(averageFeatures(removeNaN(X_test_estimated_b)), "B")
X_test_estimated_c = add_location(averageFeatures(removeNaN(X_test_estimated_c)), "C")


X_test_estimated = pd.concat([X_test_estimated_a, X_test_estimated_b,X_test_estimated_c]).rename(columns={"date_forecast" : "time"})


X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

X_test = pd.merge(X_test_estimated, X_test_targets, on=["time", "location"], how="right")

X_test = X_test.iloc[:,0:-2]
X_test["day"] = X_test["time"].dt.day_of_year
X_test["hour"] = X_test["time"].dt.hour
X_test = X_test.iloc[:,1:]
X_test["location"] = X_test["location"].map({"A": 0, "B": 1, "C": 2})
X_test = ((X_test-dataMean)/dataStd).fillna(value=0)

X_test["snow_density:kgm3"] = 0

compare = pd.concat([X_test, evalsetX])


/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="bfill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = frame.fillna(method="ffill")
/var/folders/bm/37km6rb530l8y0vv0znc0zb00000gn/T/ipykernel_69750/3652912417.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  frame = f

In [131]:
display(X_test.describe()-evalsetX.describe())

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location,day,hour
count,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.0,-1601.000000,-1.601000e+03,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000,-1601.000000
mean,1.521573,-1.261044,-0.091874,0.721157,0.719155,-0.092815,0.275442,1.546494,0.493055,0.499370,0.360873,0.365657,0.123233,-0.701595,-0.355023,-0.186837,-0.460301,-0.226857,-0.271992,0.485721,-0.528910,0.319218,0.198926,0.048791,0.394911,0.377695,-0.341711,0.317980,0.240611,0.360465,0.0,-0.156648,8.673617e-19,-0.238143,-0.001963,-0.003121,0.643898,0.409738,1.443917,0.150280,0.218347,0.089828,0.686640,-0.291306,0.046455,0.103442,0.568479,-0.000749
std,0.319175,0.156757,0.053145,0.392134,0.391666,-0.196297,-0.304053,-0.081752,0.200449,0.197907,0.375222,0.376307,-0.086919,-1.009066,-1.225272,-1.054084,-1.322855,-1.064185,-1.084564,-0.201166,-0.168538,-0.331339,0.716886,-0.205276,-0.327796,-0.322130,-1.678613,1.247988,-0.009491,-0.316279,0.0,-0.613921,8.675626e-19,-2.194844,0.292185,0.054323,-0.036357,0.467787,0.234474,-0.084450,0.107811,0.091899,-0.024375,-0.171761,-0.933963,0.011583,0.007839,-0.001980
min,0.770734,-1.858154,0.009156,0.000000,0.000000,0.001383,4.274718,1.913582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.823280,0.000000,0.000000,0.880060,0.858809,0.000000,0.000000,0.248160,0.833788,0.0,0.000000,0.000000e+00,0.000000,0.000000,-0.015356,0.911727,0.000000,1.809239,0.000000,0.009862,-0.028324,1.027784,0.292528,16.639623,0.000000,0.534488,0.000000
25%,1.318965,-1.434606,-0.034365,0.059527,0.078629,-0.008898,0.000000,1.565322,0.134428,0.159696,0.000223,0.001795,0.551402,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.058260,0.000000,0.545011,0.000000,0.000000,0.642196,0.629219,0.000000,0.000000,0.297349,0.614821,0.0,0.000000,0.000000e+00,0.000000,0.000000,-0.011172,0.617628,0.000000,1.086306,0.584275,0.003155,-0.003540,0.706043,-0.132967,0.000000,0.000000,0.561665,0.108390
50%,1.385487,-1.021303,-0.346462,0.959782,1.135299,0.049255,0.000000,1.385694,0.871392,0.804761,0.122296,0.151492,0.103066,-0.126995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.517910,0.712929,0.000000,0.000000,0.770055,0.754100,0.000000,0.000000,0.340334,0.739125,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.008300,0.629478,0.000000,1.484111,0.130705,0.278753,0.169941,0.728386,-0.319121,0.000000,0.000000,0.548077,0.072260
75%,1.828200,-0.956406,0.017895,1.231856,1.340468,-0.192946,0.000000,1.555244,0.654167,0.699174,0.792304,0.830307,0.004417,-2.285907,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.071641,0.092111,0.000000,0.000000,0.167906,0.149860,0.000000,0.000000,0.278294,0.126695,0.0,0.000000,0.000000e+00,0.000000,-0.310905,0.0

In [175]:

test_preds = pd.read_csv("sample_submission.csv")
test_preds["prediction"] = model.predict(X_test)
test_preds.to_csv("preds/NN_locationTimeInput_Ada_correctNormalization_sampleWeightedByMonth.csv", index=False)

68/68 [==============================] - 0s 3ms/step
